This code is to visualize FP, FN, and GT
* Green: True Positive
* Yellow: False Negative
* Red: False Positive



In [1]:
'''
    Green: True Positive   
    Yellow: False Negative
    Red: False Positive
'''

import os
import shutil
import numpy as np
from pathlib import Path
import cv2
from tqdm import tqdm

# Make sure file paths are correct
LABEL_DIR = Path("/home/il72/cape_town_year_of_installation/datasets/pv_capetown_after_qc_5K/3types_backup_labels/test")
PRED_DIR = Path("/home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/runs/obb/val6/labels") 
IMAGE_DIR = Path("/home/il72/cape_town_year_of_installation/datasets/pv_capetown_after_qc_5K/images/test")
OUTPUT_DIR = Path("/home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/runs/obb/val6")

IOU_THRESHOLD = 0.5

fp_dir = OUTPUT_DIR / "false_positives"
fn_dir = OUTPUT_DIR / "false_negatives"
vis_dir = OUTPUT_DIR / "vis_fp_fn"
fp_dir.mkdir(parents=True, exist_ok=True)
fn_dir.mkdir(parents=True, exist_ok=True)
vis_dir.mkdir(parents=True, exist_ok=True)

def load_boxes(path, has_conf=False):
    if not path.exists():
        print(f"Missing file: {path}")
        return np.zeros((0, 9))
    try:
        arr = np.loadtxt(path, ndmin=2)
        if has_conf and arr.shape[1] == 10:
            arr = arr[:, :9]  # drop conf
        if arr.shape[1] != 9:
            print(f"Invalid shape in {path.name}: {arr.shape}")
            return np.zeros((0, 9))
        return arr
    except Exception as e:
        print(f"Load error in {path.name}: {e}")
        return np.zeros((0, 9))

def denormalize_box(box, w, h):
    return [(box[i] * w if i % 2 == 1 else box[i] * h) for i in range(1, 9)]

def to_cv2_poly(pts):
    return np.array(pts, dtype=np.float32).reshape(-1, 1, 2)

def get_iou_poly(box1, box2, shape):
    h, w = shape
    p1 = denormalize_box(box1, w, h)
    p2 = denormalize_box(box2, w, h)
    poly1 = to_cv2_poly(p1)
    poly2 = to_cv2_poly(p2)

    if cv2.contourArea(poly1) < 0: poly1 = poly1[::-1]
    if cv2.contourArea(poly2) < 0: poly2 = poly2[::-1]

    ret, inter = cv2.intersectConvexConvex(poly1, poly2)
    if ret == 0 or inter is None:
        return 0.0
    inter_area = cv2.contourArea(inter)
    union_area = cv2.contourArea(poly1) + cv2.contourArea(poly2) - inter_area
    return inter_area / (union_area + 1e-6)

def match(gt_boxes, pred_boxes, shape, iou_thresh=0.5):
    matched_gt = set()
    matched_pred = set()
    for i, gt in enumerate(gt_boxes):
        for j, pred in enumerate(pred_boxes):
            iou = get_iou_poly(gt, pred, shape)
            if iou >= iou_thresh:
                matched_gt.add(i)
                matched_pred.add(j)
                break
    return matched_gt, matched_pred

def draw_polygon(image, box, shape, matched, label_type, conf=None):
    h, w = shape
    poly = np.array(denormalize_box(box, w, h), dtype=np.int32).reshape((-1, 1, 2))

    if label_type == "gt":
        color = (0, 255, 0) if matched else (0, 255, 255)  # green / yellow
    elif label_type == "pred":
        color = (0, 255, 0) if matched else (0, 0, 255)    # green / red
    else:
        color = (255, 255, 255)  # default white

    cv2.polylines(image, [poly], isClosed=True, color=color, thickness=2)

    # Add confidence text for predictions
    if label_type == "pred" and conf is not None:
        x, y = poly[0][0]
        text = f"{conf:.2f}"
        cv2.putText(
            image,
            text,
            (int(x), int(y) - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            thickness=1,
            lineType=cv2.LINE_AA
        )

total_fp, total_fn = 0, 0

for label_file in tqdm(list(LABEL_DIR.glob("*.txt"))):
    base = label_file.stem
    pred_file = PRED_DIR / f"{base}.txt"
    image_file = IMAGE_DIR / f"{base}.tif"
    if not image_file.exists():
        print(f"Image not found: {image_file}")
        continue

    gt = load_boxes(label_file, has_conf=False)
    pred_raw = np.loadtxt(pred_file, ndmin=2) if pred_file.exists() else np.zeros((0, 10))
    if pred_raw.shape[1] != 10:
        print(f"Invalid pred shape in {pred_file.name}: {pred_raw.shape}")
        pred_raw = np.zeros((0, 10))

    pred = pred_raw[:, :9]  # polygon coords only
    confs = pred_raw[:, 1] if len(pred_raw) > 0 else []

    image = cv2.imread(str(image_file))
    if image is None:
        print(f"Failed to read image: {image_file}")
        continue
    shape = image.shape[:2]

    matched_gt, matched_pred = match(gt, pred, shape)

    fn_count = len(gt) - len(matched_gt)
    fp_count = len(pred) - len(matched_pred)

    if fn_count > 0:
        shutil.copy(image_file, fn_dir / image_file.name)
        total_fn += 1
    if fp_count > 0:
        shutil.copy(image_file, fp_dir / image_file.name)
        total_fp += 1

    vis_img = image.copy()
    for i, gt_box in enumerate(gt):
        draw_polygon(vis_img, gt_box, shape, matched=(i in matched_gt), label_type="gt")

    for j, pred_box in enumerate(pred):
        conf = confs[j] if j < len(confs) else None
        draw_polygon(vis_img, pred_box, shape, matched=(j in matched_pred), label_type="pred", conf=conf)

    out_path = vis_dir / f"{base}.jpg"
    cv2.imwrite(str(out_path), vis_img)

print(f"\nTotal FP images: {total_fp}")
print(f"Total FN images: {total_fn}")
print(f"Visualizations saved to: {vis_dir}")


100%|██████████| 249/249 [00:17<00:00, 14.09it/s]


Total FP images: 223
Total FN images: 67
Visualizations saved to: /home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/runs/obb/val6/vis_fp_fn
